In [1]:
DATA_PATH_JSON = "./NLP-Teknofest24/NLPJSON/"
DATA_PATH_TXT = "./NLP-Teknofest24/NLPTXT-1028 Etiketlenen Raporlar/"

In [2]:
import json
d_list = list()
for i in range(1,5):
    with open(DATA_PATH_JSON+str(i)+"/all.jsonl",'r', encoding="utf-8") as f:
        for a in f.readlines():
            d_list.append(json.loads(a))

In [3]:
import pandas as pd
import numpy as np
import re
import random

In [4]:
meta_data = pd.DataFrame(d_list)

In [5]:
meta_data = meta_data.drop(["Comments","id"],axis=1)

In [6]:
meta_data

,text,label
0,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memen...,"[[32, 47, ANAT], [48, 59, ANAT], [60, 66, OBS-..."
1,BİLATERAL MAMOGRAFİ İNCELEMESİNDE; \nBilateral...,"[[36, 50, ANAT], [51, 65, ANAT], [66, 75, OBS-..."
2,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...,"[[31, 45, ANAT], [46, 60, ANAT], [61, 71, OBS-..."
3,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki memede...,"[[31, 45, ANAT], [46, 70, ANAT], [71, 78, ANAT..."
4,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki meme ...,"[[32, 44, ANAT], [45, 59, ANAT], [60, 65, OBS-..."
...,...,...
1023,Unilateral Mammografi incelemesinde;\nSağ meme...,"[[37, 45, ANAT], [45, 55, ANAT], [56, 61, OBS-..."
1024,UNİLATERAL MAMOGRAFİ İNCELEMESİ\nSağ meme cilt...,"[[32, 40, ANAT], [41, 55, ANAT], [56, 65, OBS-..."
1025,UNİLATERAL MAMOGRAFİ İNCELEMESİ\nSağ meme cild...,"[[32, 40, ANAT], [41, 46, ANAT], [47, 54, OBS-..."
1026,UNİLATERAL MAMOGRAFİ İNCELEMESİ\nSol meme cilt...,"[[32, 40, ANAT], [41, 55, ANAT], [56, 62, OBS-..."


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# First split: Train+Validation and Test
train_val_df, test_df = train_test_split(meta_data, test_size=250, random_state=42)
t
# Second split: Train and Validation
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)

In [9]:
train_df

,text,label
73,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...,"[[31, 40, ANAT], [41, 45, ANAT], [46, 60, ANAT..."
541,BİLATERAL MAMOGRAFİ İNCELEMESİ\nMeme parankimi...,"[[31, 45, ANAT], [46, 55, OBS-PRESENT], [61, 6..."
454,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memey...,"[[83, 95, ANAT], [96, 118, OBS-PRESENT], [120,..."
855,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...,"[[31, 46, ANAT], [46, 60, ANAT], [61, 71, OBS-..."
61,BİLATERAL MAMOGRAFİ İNCELEMESİ\t\nBilateral me...,"[[32, 46, ANAT], [46, 61, ANAT], [62, 71, OBS-..."
...,...,...
177,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki meme p...,"[[31, 38, ANAT], [39, 43, ANAT], [44, 60, ANAT..."
529,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki meme c...,"[[31, 43, ANAT], [44, 58, ANAT], [59, 64, OBS-..."
396,BİLATERAL MAMOGRAFİ İNCELEMESİ \nMeme paterni ...,"[[32, 44, ANAT], [45, 57, OBS-PRESENT], [58, 7..."
680,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki memede...,"[[31, 45, ANAT], [46, 63, ANAT], [64, 73, ANAT..."


In [10]:
import re

def remove_redundant_spaces_regex(text: str) -> str:
  return re.sub(r"\s+", " ", text)

In [11]:
def transform_label_normal(text: str, label: list) -> list[dict]:
    label_list_dict = []
    start : int = label[0]
    end : int = label[1]
    catg : str = label[2]
    part = text[start:end]
    token_list = remove_redundant_spaces_regex(part.strip()).split(" ")
    for i, t in enumerate(token_list):
        new_repr_inner = {"start": start, "end": start+len(t), "label": catg, "token":t}
        start += len(t) + 1
        label_list_dict.append(new_repr_inner)
    return label_list_dict

In [12]:
def transform_df_labels(df: pd.DataFrame):
    for i in range(df.shape[0]):
        text : str = df["text"].iloc[i]
        label_list : list = df["label"].iloc[i]
        transformed_labels : list = []
        for label in label_list:
            transformed_labels.extend(transform_label_normal(text,label))
        df["label"].iloc[i] = transformed_labels
    return df

In [13]:
train_df_copy = train_df.copy()
train_df_copy = transform_df_labels(train_df_copy)
test_df_copy = test_df.copy()
test_df_copy = transform_df_labels(test_df_copy)
val_df_copy = val_df.copy()
val_df_copy = transform_df_labels(val_df_copy)
train_df_copy

,text,label
73,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...,"[{'start': 31, 'end': 40, 'label': 'ANAT', 'to..."
541,BİLATERAL MAMOGRAFİ İNCELEMESİ\nMeme parankimi...,"[{'start': 31, 'end': 35, 'label': 'ANAT', 'to..."
454,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memey...,"[{'start': 83, 'end': 86, 'label': 'ANAT', 'to..."
855,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...,"[{'start': 31, 'end': 40, 'label': 'ANAT', 'to..."
61,BİLATERAL MAMOGRAFİ İNCELEMESİ\t\nBilateral me...,"[{'start': 32, 'end': 41, 'label': 'ANAT', 'to..."
...,...,...
177,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki meme p...,"[{'start': 31, 'end': 34, 'label': 'ANAT', 'to..."
529,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki meme c...,"[{'start': 31, 'end': 34, 'label': 'ANAT', 'to..."
396,BİLATERAL MAMOGRAFİ İNCELEMESİ \nMeme paterni ...,"[{'start': 32, 'end': 36, 'label': 'ANAT', 'to..."
680,BİLATERAL MAMOGRAFİ İNCELEMESİ\nHer iki memede...,"[{'start': 31, 'end': 34, 'label': 'ANAT', 'to..."


In [14]:
label_mapping = {
    'O': 0,
    'ANAT': 1,
    'OBS-PRESENT': 2,
    'OBS-UNCERTAIN':3,
    'OBS-ABSENT': 4,
    'IMPRESSION': 5,
}

In [15]:
import nltk
t = nltk.tokenize.NLTKWordTokenizer()
def transform_into_ner_tags(df: pd.DataFrame):
    for i in range(df.shape[0]):
        labels = df["label"].iloc[i]
        text = df["text"].iloc[i]
        count = 0
        ner_tags = []
        tokenized_text = remove_redundant_spaces_regex(text.replace("\xa0"," ").replace("\n"," ")).split()#t.tokenize(text)
        #if i==0:print(tokenized_text)
        for token in tokenized_text:
            #if i==0:print("token: ",token, "  label:",labels[count]["token"], "   count:",count)
            if count < len(labels) and (labels[count]["token"] in token or labels[count]["token"] == token or token in labels[count]["token"]):
                ner_tags.append(label_mapping[labels[count]["label"]])
                count+=1
            else: ner_tags.append(label_mapping["O"])
        df.iloc[i] = tokenized_text, ner_tags
    df.columns = ["tokens","ner_tags"]

    return df

In [16]:
train_df_copy = transform_into_ner_tags(train_df_copy)
test_df_copy = transform_into_ner_tags(test_df_copy)
val_df_copy = transform_into_ner_tags(val_df_copy)

In [17]:
def remove_impression_tags(l : list) -> list:
    for i in range(len(l)):
        if l[i] == 5:
            l[i] = 0
    return l


In [18]:
train_df_copy["ner_tags"].apply(remove_impression_tags)
test_df_copy["ner_tags"].apply(remove_impression_tags)
val_df_copy["ner_tags"].apply(remove_impression_tags)

919    [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
159    [0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...
607    [0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, ...
674    [0, 0, 0, 1, 1, 2, 2, 2, 0, 0, 1, 1, 2, 2, 2, ...
749    [0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, ...
                             ...                        
728    [0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 1, 1, 0, 1, 1, ...
540    [0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...
274    [0, 0, 0, 1, 1, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1, ...
521    [0, 0, 0, 1, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 2, ...
711    [0, 0, 0, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 2, 2, ...
Name: ner_tags, Length: 195, dtype: object

In [112]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/convbert-base-turkish-cased")

In [114]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [24]:
from datasets import load_from_disk
dataset = load_from_disk("ner_dataset_conll_format_impression_yok")

In [115]:
tokenized_dataset = dataset.map(tokenize_and_align_labels,batched=True)

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


Map:   0%|          | 0/583 [00:00<?, ? examples/s]

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [116]:
len(tokenized_dataset["test"]["input_ids"][0])

139

In [73]:
out_list = []
obs_p_list = []
obs_a_list = []
obs_u_list = []
anat_list = []
for i in range(tokenized_dataset["train"].shape[0]):
    txt = tokenized_dataset["train"]["input_ids"][i]
    lbl = tokenized_dataset["train"]["labels"][i]
    for ind,l in enumerate(lbl):
        a = txt[ind]
        if l == 2:
            obs_p_list.append(a)
        elif l == 3:
            obs_u_list.append(a)
        elif l == 4:
            obs_a_list.append(a)
        elif l == 1:
            anat_list.append(a)
        elif l == 0:
            out_list.append(a)

In [74]:
obsp_df = pd.DataFrame(pd.DataFrame(obs_p_list).value_counts().reset_index())
obsa_df = pd.DataFrame(pd.DataFrame(obs_a_list).value_counts().reset_index())
obsu_df = pd.DataFrame(pd.DataFrame(obs_u_list).value_counts().reset_index())
anat_df = pd.DataFrame(pd.DataFrame(anat_list).value_counts().reset_index())
out_df = pd.DataFrame(pd.DataFrame(out_list).value_counts().reset_index())

In [75]:
obsp_df.columns = ["words","count"]
obsu_df.columns = ["words","count"]
obsa_df.columns = ["words","count"]
anat_df.columns = ["words","count"]
out_df.columns = ["words","count"]

In [76]:
all_tags_list = []
all_tags_list.extend(obs_p_list)
all_tags_list.extend(obs_u_list)
all_tags_list.extend(obs_a_list)
all_tags_list.extend(anat_list)
all_tags_list.extend(out_list)

In [77]:
len(all_tags_list)

80116

In [117]:
hardcoded_dict = dict()
for i in all_tags_list:
    total_count = all_tags_list.count(i)
    if i not in hardcoded_dict and total_count >= 5:
        OC = out_list.count(i)/total_count
        ANC = anat_list.count(i)/total_count
        PC = obs_p_list.count(i)/total_count
        UC = obs_u_list.count(i)/total_count
        AC = obs_a_list.count(i)/total_count
        
        hardcoded_dict[i] = [OC,ANC,PC,UC,AC]
        


In [118]:
hardcoded_probs_df = pd.DataFrame(hardcoded_dict).T.reset_index()
hardcoded_probs_df.columns=["token","O","ANAT","OBS-PRESENT","OBS-UNCERTAIN","OBS-ABSENT"]

In [55]:
#hardcoded_probs_df = hardcoded_probs_df.sort_values("total_count",ascending=False)

In [80]:
hardcoded_probs_df = hardcoded_probs_df.reset_index().drop("index",axis=1)

In [81]:
hardcoded_probs_df

,token,O,ANAT,OBS-PRESENT,OBS-UNCERTAIN,OBS-ABSENT
0,7419,0.291560,0.020460,0.332481,0.000000,0.355499
1,6570,0.236559,0.000000,0.763441,0.000000,0.000000
2,2067,0.113712,0.000000,0.886288,0.000000,0.000000
3,18,0.767349,0.008817,0.213026,0.004835,0.005973
4,7085,0.156134,0.000000,0.769517,0.000000,0.074349
...,...,...,...,...,...,...
427,5129,1.000000,0.000000,0.000000,0.000000,0.000000
428,20111,1.000000,0.000000,0.000000,0.000000,0.000000
429,3573,1.000000,0.000000,0.000000,0.000000,0.000000
430,4029,1.000000,0.000000,0.000000,0.000000,0.000000


In [70]:
combined_test_preds_proba = np.load("combined_test_preds_proba.npy")

In [120]:
import pickle

# Save the dictionary to a file using pickle
with open('hardcoded_dict_tokenized_5.pkl', 'wb') as f:
    pickle.dump(hardcoded_dict, f)

In [103]:
import pickle
with open('hardcoded_dict_tokenized.pkl', 'rb') as f:
    hardcoded_dict = pickle.load(f)
print(hardcoded_dict)

{7419: [0.2915601023017903, 0.020460358056265986, 0.33248081841432225, 0.0, 0.3554987212276215], 6570: [0.23655913978494625, 0.0, 0.7634408602150538, 0.0, 0.0], 2067: [0.11371237458193979, 0.0, 0.8862876254180602, 0.0, 0.0], 18: [0.767349260523322, 0.00881683731513083, 0.21302616609783845, 0.004835039817974972, 0.005972696245733789], 7085: [0.15613382899628253, 0.0, 0.7695167286245354, 0.0, 0.07434944237918216], 4082: [0.17079889807162535, 0.0, 0.5454545454545454, 0.0027548209366391185, 0.2809917355371901], 20471: [0.058823529411764705, 0.0, 0.9411764705882353, 0.0, 0.0], 2235: [0.13871951219512196, 0.0, 0.5945121951219512, 0.0, 0.26676829268292684], 2462: [0.15738284703801944, 0.01856763925729443, 0.47745358090185674, 0.0, 0.34659593280282935], 8038: [0.14563106796116504, 0.000970873786407767, 0.5174757281553398, 0.0, 0.3359223300970874], 19943: [0.13675213675213677, 0.002849002849002849, 0.8319088319088319, 0.0, 0.02849002849002849], 6915: [0.9505119453924915, 0.008532423208191127, 0

In [104]:
count = 0
for i1, text in enumerate(tokenized_dataset["test"]["input_ids"][:2]):
    for i2, id in enumerate(text):
        if id in hardcoded_dict:
            print(combined_test_preds_proba[i1][i2]*0.5 + np.array(hardcoded_dict[id])*0.5)
            count+=1
        else:
            print("combined ==> ",combined_test_preds_proba[i1][i2])

combined ==>  [0.6789609  0.05898926 0.2094927  0.01899804 0.03355912]
[9.99773771e-01 1.33824709e-04 3.26089757e-05 1.85718127e-05
 4.12225490e-05]
[9.99791056e-01 1.22662634e-04 3.11216427e-05 1.71160609e-05
 3.80616038e-05]
[9.99790817e-01 1.25801453e-04 2.82014425e-05 1.71468855e-05
 3.80873607e-05]
[9.99788493e-01 1.28123982e-04 2.79114593e-05 1.67156741e-05
 3.87952641e-05]
[9.99772280e-01 1.38421354e-04 2.96012367e-05 1.83229622e-05
 4.13205453e-05]
[9.99782950e-01 1.32811052e-04 2.77276922e-05 1.76117355e-05
 3.89120723e-05]
[9.99770284e-01 1.41033976e-04 3.08479393e-05 1.75626355e-05
 4.02760634e-05]
[9.99758005e-01 1.49333689e-04 3.16149526e-05 1.89989223e-05
 4.19770186e-05]
[9.98892473e-01 1.01645543e-03 3.12253178e-05 1.84415167e-05
 4.14275019e-05]
[9.98789320e-01 1.10456121e-03 3.61275052e-05 2.11447932e-05
 4.88748265e-05]
[9.98790803e-01 1.10944236e-03 3.50377522e-05 1.99176793e-05
 4.48292631e-05]
[9.98740440e-01 1.14898055e-03 4.00317949e-05 2.18939622e-05
 4.8575915

In [96]:
combined_test_preds_proba[0][0]

array([0.6789609 , 0.05898926, 0.2094927 , 0.01899804, 0.03355912],
      dtype=float32)

In [63]:
hardcoded_probs_df.to_csv("tokenized_hardcoded_probs_df.csv")